# 03a - pois_filter_salud

This notebook __loads__ health facilities in Santiago, Chile, __filters__ for relevant data and creates __2 files (private, public) for each of the following 4 categories__:

1. Hospitals
2. Clinics
3. Vaccination centers
4. Optician, audiologist, dentist

## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

In [2]:
##### WARNING ##### WARNING ##### WARNING ##### WARNING ##### WARNING #####

# save output to local?
save = True

##### WARNING ##### WARNING ##### WARNING ##### WARNING ##### WARNING #####

## Load data

In [3]:
establec_salud_dir = "../../../data/external/temporal_todocker/santiago/proximidad/establec_salud_32719.gpkg"
establec_salud = gpd.read_file(establec_salud_dir)
establec_salud = establec_salud.to_crs("EPSG:4326")

# Show
print(establec_salud.shape)
establec_salud.head(2)

(874, 39)


,objectid,cod_region,nom_region,cod_provin,nom_provin,cod_comuna,nom_comuna,c_ant,c_vig,c_mad,...,tipo_camb,latitud,longitud,prestador,estado,nivel_com,modalidad,coord_x,coord_y,geometry
0,1,13,Metropolitana de Santiago,131,Santiago,13108,Independencia,09-030,109030.0,No Aplica,...,Sin cambio,-33.429013,-70.655438,PÃºblico,Vigente en operaciÃ³n,Baja Complejidad,AtenciÃ³n Abierta-Ambulatoria,346100.148329,6.299926e+06,POINT (-70.65544 -33.42901)
1,2,13,Metropolitana de Santiago,131,Santiago,13101,Santiago,10-030,110030.0,No Aplica,...,Sin cambio,-33.448174,-70.669444,PÃºblico,Vigente en operaciÃ³n,Baja Complejidad,AtenciÃ³n Abierta-Ambulatoria,344832.041446,6.297780e+06,POINT (-70.66944 -33.44817)


## Data exploration and filtering (Applies to all desired categories)

#### __Exploration__ - Column 'ambito'

In [4]:
# --> Keep 'Unidad de AtenciÃ³n' (Care unit) y 'Establecimiento de Salud' (Health facility) because:

# 'Establecimiento No de Salud': There's just one and has as type "Instituto de Seguridad Laboral" and name "Programa COVID19 -ISL'
# 'Establecimiento Medicina Legal': There's just two and have as type "Servicio Médico Legal", from Ministerio de Justicia.
# 'Programa de AtenciÃ³n': has PRAIS (PROGRAMA DE REPARACIÓN Y ATENCIÓN INTEGRAL EN SALUD Y DERECHOS HUMANOS).
#     There are 4 in Santiago, all near 'Unidad de AtenciÃ³n' and/or'Establecimiento de Salud'.

establec_salud.ambito.unique()

array(['Unidad de AtenciÃ³n', 'Programa de AtenciÃ³n',
       'Establecimiento de Salud', 'Establecimiento Medicina Legal',
       'Establecimiento No de Salud'], dtype=object)

#### __Exploration__ - Column 'prestador'

In [5]:
# Keep 'PÃºblico' (Public) and 'Privado' (Private), the rest are not for general public.
establec_salud.prestador.unique()

array(['PÃºblico', 'Privado', 'Fuerzas Armadas y de Orden',
       'GendarmerÃ\xada', 'Pendiente', 'InstituciÃ³n Medicina Legal'],
      dtype=object)

#### __Filter__ for columns 'ambito' and 'prestador'

In [6]:
ambito_list = ['Unidad de AtenciÃ³n','Establecimiento de Salud']
idx_1 = establec_salud.ambito.isin(ambito_list)

prestador_list = ['PÃºblico', 'Privado']
idx_2 = establec_salud.prestador.isin(prestador_list)

establec_salud_f1 = establec_salud.loc[(idx_1)&(idx_2)].copy()

# Show
print(establec_salud_f1.shape)
establec_salud_f1.head(2)

(838, 39)


,objectid,cod_region,nom_region,cod_provin,nom_provin,cod_comuna,nom_comuna,c_ant,c_vig,c_mad,...,tipo_camb,latitud,longitud,prestador,estado,nivel_com,modalidad,coord_x,coord_y,geometry
0,1,13,Metropolitana de Santiago,131,Santiago,13108,Independencia,09-030,109030.0,No Aplica,...,Sin cambio,-33.429013,-70.655438,PÃºblico,Vigente en operaciÃ³n,Baja Complejidad,AtenciÃ³n Abierta-Ambulatoria,346100.148329,6.299926e+06,POINT (-70.65544 -33.42901)
1,2,13,Metropolitana de Santiago,131,Santiago,13101,Santiago,10-030,110030.0,No Aplica,...,Sin cambio,-33.448174,-70.669444,PÃºblico,Vigente en operaciÃ³n,Baja Complejidad,AtenciÃ³n Abierta-Ambulatoria,344832.041446,6.297780e+06,POINT (-70.66944 -33.44817)


#### __Exploration__ - Column 'nivel' (Not useful for filtering/categorizing data)

In [7]:
establec_salud_f1.nivel.unique()

array(['Secundario', 'No Aplica', 'Primario', 'Terciario',
       'Primario-Secundario ( Por confirmar)', 'terciario'], dtype=object)

#### __Exploration__ - Column 'modalidad' (Not useful for filtering/categorizing data)

In [8]:
# Exploration - Is 'modalidad' useful for categorizing hospitals?
# Answer No. But it was discovered that column 'tipo' is useful.
# By filtering the full original df we get the following results:

# Modalidad 'AtenciÃ³n Abierta-Ambulatoria' (Open ambulatory care)
    # 'nivel': 218 'No Aplica', 430 'Primario', 77 'Secundario' y 2 'Terciario'
    # LOTS and LOTS of 'tipo' (type)

# Modalidad AtenciÃ³n Cerrada-Hospitalaria' (Closed hospitable? care)
    # 'nivel': 48 'No Aplica', 2 'Primario-Secundario ( Por confirmar)', 1 'terciario' y 60 'Terciario'
    # 'tipo': 46 'hospital', 58 'ClÃ\xadnica', 4 'Centro de Salud', 1 'Centro de Referencia de Salud (CRS)', 1 'Centro CorporaciÃ³n para la NutriciÃ³n Infantil (CONIN)',4 'Dispositivo Incorporado por Crisis Sanitaria'

# Modalidad 'No Aplica'
# 'nivel' 31 'No Aplica'
# 'tipo' 27 'Laboratorio ClÃ\xadnico o Dental', 1 'Hospital Digital', 2 'Servicio Médico Legal' y 1 'Instituto de Seguridad Laboral'

establec_salud_f1.modalidad.unique()

array(['AtenciÃ³n Abierta-Ambulatoria', 'AtenciÃ³n Cerrada-Hospitalaria',
       'No Aplica'], dtype=object)

#### __Exploration__ - Column 'tipo'

In [9]:
# Research available to team in Miro.

# DO NOT KEEP:
# [For goverment employees] 'Unidad de Salud Funcionarios'
# [Service] DirecciÃ³n Servicio de Salud'
# [Program] 'Programa de ReparaciÃ³n y AtenciÃ³n Integral de Salud' (Se descartó en 'ambito')
# [Movil] 'ClÃ\xadnica Dental MÃ³vil'
# [N/A] 'Oficina Sanitaria'
# [Mental health]'Centro Comunitario de Salud Mental (COSAM)'
# [Mental health]'Consultorio de Salud Mental'
# [Must go to another facility first] 'Centro de Referencia de Salud (CRS)'
# [Other] 'Centro CorporaciÃ³n para la NutriciÃ³n Infantil (CONIN)'
# [Blood, tissues] 'Consultorio de DiagnÃ³stico y Tratamiento (CDT)'
# [Mental health] 'Unidad de Salud Mental'
# [Other] 'Hospital Digital'
# [Call center] 'Centro de RegulaciÃ³n MÃ©dica de las Urgencias (SAMU)'
# [Justice Department] 'Servicio MÃ©dico Legal'
# [Other] 'Dispositivo Incorporado por Crisis Sanitaria'
# [Special health service] 'Centro de DiÃ¡lisis'
# [Other] 'Pendiente'
# [Other] 'Instituto de Seguridad Laboral'

# KEEP:
tipo_list = ['Hospital', #Hospital
             'ClÃ\xadnica', #Clinic
             'Centro de Salud', #Clinic
             'Centro MÃ©dico y Dental', #Clinic, ado (Audiologist, Dentist, Optitian)
             'Centro de Salud Familiar (CESFAM)', #Clinic
             'Posta de Salud Rural (PSR)', #Clinic
             'Centro Comunitario de Salud Familiar (CECOSF)', #Clinic
             'Servicio de AtenciÃ³n Primaria de Urgencia de Alta Resolutividad (SAR)', #Clinic
             'Servicio de AtenciÃ³n Primaria de Urgencia (SAPU)', #Clinic
             'Laboratorio ClÃ\xadnico o Dental', #ado
             'Vacunatorio', #Vaccination center
             'ClÃ\xadnica Dental', #ado
             'Consultorio General Rural (CGR)', #Clinic 
             'Servicio de Urgencia Rural (SUR)', #Clinic
             'Centro OftalmolÃ³gico'] #ado

establec_salud.tipo.unique()

array(['Unidad de Salud Funcionarios', 'DirecciÃ³n Servicio de Salud',
       'Programa de ReparaciÃ³n y AtenciÃ³n Integral de Salud',
       'ClÃ\xadnica Dental MÃ³vil', 'Hospital', 'ClÃ\xadnica',
       'Centro de Salud', 'Centro MÃ©dico y Dental',
       'Centro de Salud Familiar (CESFAM)', 'Posta de Salud Rural (PSR)',
       'Centro Comunitario de Salud Mental  (COSAM)',
       'Centro Comunitario de Salud Familiar (CECOSF)',
       'Servicio de AtenciÃ³n Primaria de Urgencia de Alta Resolutividad (SAR)',
       'Servicio de AtenciÃ³n Primaria de Urgencia (SAPU)',
       'Centro de Referencia de Salud (CRS)',
       'Laboratorio ClÃ\xadnico o Dental', 'Vacunatorio',
       'Centro CorporaciÃ³n para la NutriciÃ³n Infantil (CONIN)',
       'ClÃ\xadnica Dental',
       'Consultorio de DiagnÃ³stico y Tratamiento (CDT)',
       'Consultorio General Rural (CGR)', 'Unidad de Salud Mental',
       'Hospital Digital',
       'Centro de RegulaciÃ³n MÃ©dica de las Urgencias (SAMU)',
       '

In [10]:
idx_3 = establec_salud_f1.tipo.isin(tipo_list)

establec_salud_f2 = establec_salud_f1.loc[idx_3].copy()

# Show
print(establec_salud_f2.shape)
establec_salud_f2.head(2)

(763, 39)


,objectid,cod_region,nom_region,cod_provin,nom_provin,cod_comuna,nom_comuna,c_ant,c_vig,c_mad,...,tipo_camb,latitud,longitud,prestador,estado,nivel_com,modalidad,coord_x,coord_y,geometry
9,10,13,Metropolitana de Santiago,131,Santiago,13108,Independencia,09-100,109100.0,No Aplica,...,Sin cambio,-33.416586,-70.653318,PÃºblico,Vigente en operaciÃ³n,Alta Complejidad,AtenciÃ³n Cerrada-Hospitalaria,346275.350466,6.301307e+06,POINT (-70.65332 -33.41659)
10,11,13,Metropolitana de Santiago,131,Santiago,13108,Independencia,09-101,109101.0,No Aplica,...,Sin cambio,-33.415335,-70.655086,PÃºblico,Vigente en operaciÃ³n,Alta Complejidad,AtenciÃ³n Cerrada-Hospitalaria,346108.737232,6.301443e+06,POINT (-70.65509 -33.41534)


## Pois categorization and saving

#### __pois categorization__ - Hospitals (code: hospital)

In [11]:
# -------------------------------------------------------------------------------------------------- Hospital (private)

# Filter
idx = (establec_salud_f2['prestador']=='Privado') & (establec_salud_f2['tipo']=='Hospital')
hospital_privado = establec_salud_f2.loc[idx].copy()
# Format
hospital_privado['code'] = 'hospital_priv'
hospital_privado = hospital_privado[['code','geometry']]
print(hospital_privado.shape)
#Save
if save:
    hospital_privado.to_file('../../../data/external/temporal_fromjupyter/santiago/pois/hospital_priv.gpkg', driver='GPKG')

# -------------------------------------------------------------------------------------------------- Hospital (public)

# Filter
idx = (establec_salud_f2['prestador']=='PÃºblico') & (establec_salud_f2['tipo']=='Hospital')
hospital_publico = establec_salud_f2.loc[idx].copy()
# Format
hospital_publico['code'] = 'hospital_pub'
hospital_publico = hospital_publico[['code','geometry']]
print(hospital_publico.shape)
#Save
if save:
    hospital_publico.to_file('../../../data/external/temporal_fromjupyter/santiago/pois/hospital_pub.gpkg', driver='GPKG')

(6, 2)
(33, 2)


#### __pois categorization__ - Clinics (code: clinica)

In [12]:
#Clinics
clinics_list = ['ClÃ\xadnica', #Clinic
                'Centro de Salud', #Clinic
                'Centro MÃ©dico y Dental', #Clinic, ado
                'Centro de Salud Familiar (CESFAM)', #Clinic
                'Posta de Salud Rural (PSR)', #Clinic
                'Centro Comunitario de Salud Familiar (CECOSF)', #Clinic
                'Servicio de AtenciÃ³n Primaria de Urgencia de Alta Resolutividad (SAR)', #Clinic
                'Servicio de AtenciÃ³n Primaria de Urgencia (SAPU)', #Clinic
                'Consultorio General Rural (CGR)', #Clinic 
                'Servicio de Urgencia Rural (SUR)'] #Clinic

# -------------------------------------------------------------------------------------------------- Clinic (private)

# Filter
idx = (establec_salud_f2['prestador']=='Privado') & (establec_salud_f2['tipo'].isin(clinics_list))
clinica_privada = establec_salud_f2.loc[idx].copy()
# Format
clinica_privada['code'] = 'clinica_priv'
clinica_privada = clinica_privada[['code','geometry']]
print(clinica_privada.shape)
#Save
if save:
    clinica_privada.to_file('../../../data/external/temporal_fromjupyter/santiago/pois/clinica_priv.gpkg', driver='GPKG')

# -------------------------------------------------------------------------------------------------- Clinic (public)

# Filter
idx = (establec_salud_f2['prestador']=='PÃºblico') & (establec_salud_f2['tipo'].isin(clinics_list))
clinica_publica = establec_salud_f2.loc[idx].copy()
# Format
clinica_publica['code'] = 'clinica_pub'
clinica_publica = clinica_publica[['code','geometry']]
print(clinica_publica.shape)
#Save
if save:
    clinica_publica.to_file('../../../data/external/temporal_fromjupyter/santiago/pois/clinica_pub.gpkg', driver='GPKG')

(241, 2)
(402, 2)


#### __pois categorization__ - Vaccination centers (code: vacunatorio)

In [13]:
# -------------------------------------------------------------------------------------------------- # Vaccination centers (private)

# Filter
idx = (establec_salud_f2['prestador']=='Privado') & (establec_salud_f2['tipo']=='Vacunatorio')
vacunatorio_privado = establec_salud_f2.loc[idx].copy()
# Format
vacunatorio_privado['code'] = 'vacunatorio_priv'
vacunatorio_privado = vacunatorio_privado[['code','geometry']]
print(vacunatorio_privado.shape)
#Save
if save:
    vacunatorio_privado.to_file('../../../data/external/temporal_fromjupyter/santiago/pois/vacunatorio_priv.gpkg', driver='GPKG')

# -------------------------------------------------------------------------------------------------- # Vaccination centers (public)

# Filter
idx = (establec_salud_f2['prestador']=='PÃºblico') & (establec_salud_f2['tipo']=='Vacunatorio')
vacunatorio_publico = establec_salud_f2.loc[idx].copy()
# Format
vacunatorio_publico['code'] = 'vacunatorio_pub'
vacunatorio_publico = vacunatorio_publico[['code','geometry']]
print(vacunatorio_publico.shape)
#Save
if save:
    vacunatorio_publico.to_file('../../../data/external/temporal_fromjupyter/santiago/pois/vacunatorio_pub.gpkg', driver='GPKG')

(20, 2)
(1, 2)


#### __pois categorization__ - Audiologist, dentist, optician (code: consultorio_ado)

In [14]:
ado_list = ['Centro MÃ©dico y Dental', #Clínica, ado
            'Laboratorio ClÃ\xadnico o Dental', #ado
            'ClÃ\xadnica Dental', #ado
            'Centro OftalmolÃ³gico'] #ado

# -------------------------------------------------------------------------------------------------- # Audiologist, dentist, optician (private)

# Filter
idx = (establec_salud_f2['prestador']=='Privado') & (establec_salud_f2['tipo'].isin(ado_list))
consultorio_ado_privado = establec_salud_f2.loc[idx].copy()
# Format
consultorio_ado_privado['code'] = 'consult_ado_priv'
consultorio_ado_privado = consultorio_ado_privado[['code','geometry']]
print(consultorio_ado_privado.shape)
#Save
if save:
    consultorio_ado_privado.to_file('../../../data/external/temporal_fromjupyter/santiago/pois/consult_ado_priv.gpkg', driver='GPKG')

# -------------------------------------------------------------------------------------------------- # Audiologist, dentist, optician (public)

# Filter
idx = (establec_salud_f2['prestador']=='PÃºblico') & (establec_salud_f2['tipo'].isin(ado_list))
consultorio_ado_publico = establec_salud_f2.loc[idx].copy()
# Format
consultorio_ado_publico['code'] = 'consult_ado_pub'
consultorio_ado_publico = consultorio_ado_publico[['code','geometry']]
print(consultorio_ado_publico.shape)
#Save
if save:
    consultorio_ado_publico.to_file('../../../data/external/temporal_fromjupyter/santiago/pois/consult_ado_pub.gpkg', driver='GPKG')

(80, 2)
(3, 2)


# Old categorization method (ignore)

In [68]:
#Hospitals (private)
establec_salud_f2['hospital_privado'] = 0
idx = (establec_salud_f2['prestador']=='Privado') & (establec_salud_f2['tipo']=='Hospital')
establec_salud_f2.loc[idx, 'hospital_privado'] = 1

#Hospitals (public)
establec_salud_f2['hospital_publico'] = 0
idx = (establec_salud_f2['prestador']=='PÃºblico') & (establec_salud_f2['tipo']=='Hospital')
establec_salud_f2.loc[idx, 'hospital_publico'] = 1

# --------------------------------------------------------------------------------------------------
#Clinics
clinics_list = ['ClÃ\xadnica', #Clínica
                'Centro de Salud', #Clínica
                'Centro MÃ©dico y Dental', #Clínica, triple O
                'Centro de Salud Familiar (CESFAM)', #Clínica
                'Posta de Salud Rural (PSR)', #Clínica
                'Centro Comunitario de Salud Familiar (CECOSF)', #Clínica
                'Servicio de AtenciÃ³n Primaria de Urgencia de Alta Resolutividad (SAR)', #Clínica
                'Servicio de AtenciÃ³n Primaria de Urgencia (SAPU)', #Clínica
                'Consultorio General Rural (CGR)', #Clínica 
                'Servicio de Urgencia Rural (SUR)'] #Clínica

#Clinics (private)
establec_salud_f2['clinica_privada'] = 0
idx = (establec_salud_f2['prestador']=='Privado') & (establec_salud_f2['tipo'].isin(clinics_list))
establec_salud_f2.loc[idx, 'clinica_privada'] = 1

#Clinics (public)
establec_salud_f2['clinica_publica'] = 0
idx = (establec_salud_f2['prestador']=='PÃºblico') & (establec_salud_f2['tipo'].isin(clinics_list))
establec_salud_f2.loc[idx, 'clinica_publica'] = 1

# --------------------------------------------------------------------------------------------------
#Optician, audiologist, dentist (special)
special_list = ['Centro MÃ©dico y Dental', #Clínica, triple O
                'Laboratorio ClÃ\xadnico o Dental', #Triple O
                'ClÃ\xadnica Dental', #Triple O
                'Centro OftalmolÃ³gico'] #Triple O

#Optician, audiologist, dentist (private)
establec_salud_f2['especial_privada'] = 0
idx = (establec_salud_f2['prestador']=='Privado') & (establec_salud_f2['tipo'].isin(special_list))
establec_salud_f2.loc[idx, 'especial_privada'] = 1

#Optician, audiologist, dentist (public)
establec_salud_f2['especial_publica'] = 0
idx = (establec_salud_f2['prestador']=='PÃºblico') & (establec_salud_f2['tipo'].isin(special_list))
establec_salud_f2.loc[idx, 'especial_publica'] = 1

# --------------------------------------------------------------------------------------------------
#Vaccines (private)
establec_salud_f2['vacunas_privada'] = 0
idx = (establec_salud_f2['prestador']=='Privado') & (establec_salud_f2['tipo']=='Vacunatorio')
establec_salud_f2.loc[idx, 'vacunas_privada'] = 1

#Vaccines (public)
establec_salud_f2['vacunas_publica'] = 0
idx = (establec_salud_f2['prestador']=='PÃºblico') & (establec_salud_f2['tipo']=='Vacunatorio')
establec_salud_f2.loc[idx, 'vacunas_publica'] = 1

# Show
print(establec_salud_f2.shape)
establec_salud_f2.head(10)

(763, 47)


,objectid,cod_region,nom_region,cod_provin,nom_provin,cod_comuna,nom_comuna,c_ant,c_vig,c_mad,...,coord_y,geometry,hospital_privado,hospital_publico,clinica_privada,clinica_publica,especial_privada,especial_publica,vacunas_privada,vacunas_publica
9,10,13,Metropolitana de Santiago,131,Santiago,13108,Independencia,09-100,109100.0,No Aplica,...,6.301307e+06,POINT (-70.65332 -33.41659),0,1,0,0,0,0,0,0
10,11,13,Metropolitana de Santiago,131,Santiago,13108,Independencia,09-101,109101.0,No Aplica,...,6.301443e+06,POINT (-70.65509 -33.41534),0,1,0,0,0,0,0,0
11,12,13,Metropolitana de Santiago,131,Santiago,13127,Recoleta,09-102,109102.0,No Aplica,...,6.300666e+06,POINT (-70.65054 -33.42240),0,1,0,0,0,0,0,0
12,13,13,Metropolitana de Santiago,131,Santiago,13108,Independencia,09-103,109103.0,No Aplica,...,6.301138e+06,POINT (-70.65211 -33.41813),0,1,0,0,0,0,0,0
13,14,13,Metropolitana de Santiago,133,Chacabuco,13303,Tiltil,09-104,109104.0,No Aplica,...,6.337444e+06,POINT (-70.92452 -33.08679),0,1,0,0,0,0,0,0
14,15,13,Metropolitana de Santiago,131,Santiago,13108,Independencia,09-200,109200.0,No Aplica,...,6.300922e+06,POINT (-70.65312 -33.42006),1,0,0,0,0,0,0,0
15,16,13,Metropolitana de Santiago,131,Santiago,13127,Recoleta,09-201,109201.0,No Aplica,...,6.300131e+06,POINT (-70.64854 -33.42726),0,0,1,0,0,0,0,0
16,17,13,Metropolitana de Santiago,131,Santiago,13107,Huechuraba,09-202,109202.0,No Aplica,...,6.306833e+06,POINT (-70.67849 -33.36641),0,0,1,0,0,0,0,0
17,18,13,Metropolitana de Santiago,131,Santiago,13125,Quilicura,09-204,109204.0,No Aplica,...,6.306965e+06,POINT (-70.70183 -33.36491),0,0,1,0,0,0,0,0
18,19,13,Metropolitana de Santiago,131,Santiago,13125,Quilicura,09-205,109205.0,No Aplica,...,6.308000e+06,POINT (-70.72818 -33.35521),0,0,1,0,0,0,0,0


In [71]:
test = establec_salud_f2.copy()

# Test how many categories were assigned to each row
categories = ['hospital_privado','hospital_publico',
              'clinica_privada','clinica_publica',
              'especial_privada','especial_publica',
              'vacunas_privada','vacunas_publica']

# All rows should have 1 category only except 'Centro MÃ©dico y Dental', which should have 2. None should have 0.
test['id_sum_test'] = test[categories].sum(axis=1)
print(test.id_sum_test.unique())
# Rows that have 2 categories:
print(test.loc[test['id_sum_test']==2].tipo.unique())

[1 2]
['Centro MÃ©dico y Dental']


In [73]:
pois_salud = establec_salud_f2[categories+['geometry']]

# Show
print(pois_salud.shape)
pois_salud.head(2)

(763, 9)


,hospital_privado,hospital_publico,clinica_privada,clinica_publica,especial_privada,especial_publica,vacunas_privada,vacunas_publica,geometry
9,0,1,0,0,0,0,0,0,POINT (-70.65332 -33.41659)
10,0,1,0,0,0,0,0,0,POINT (-70.65509 -33.41534)
